# f) Ngonye Falls Power and Energy Periods



## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|



## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|



## Parameters

In [1]:
input_data='./input_data/'
output_data='./output_data/'

In [4]:
debt_period=18
equity_period=25

## Libraries and Setup

In [2]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

In [3]:
yearly=pd.read_csv(output_data + "rec1_p_and_e_yearly.csv")

In [17]:
yearly['WaterYear'].max()

2016

In [18]:

sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims
#yearly

,StartYear,DebtEnd,EquityEnd
0,1924,1941,1948
1,1925,1942,1949
2,1926,1943,1950
3,1927,1944,1951
4,1928,1945,1952
...,...,...,...
64,1988,2005,2012
65,1989,2006,2013
66,1990,2007,2014
67,1991,2008,2015


In [35]:
pd.concat([sims,pd.DataFrame(columns = list(map(lambda x : 'y'+str(x), range(1,equity_period+1,1))))])


,StartYear,DebtEnd,EquityEnd,y1,y2,y3,y4,y5,y6,y7,...,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25
0,1924.0,1941.0,1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1925.0,1942.0,1949.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1926.0,1943.0,1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1927.0,1944.0,1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1928.0,1945.0,1952.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1988.0,2005.0,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,1989.0,2006.0,2013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1990.0,2007.0,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,1991.0,2008.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['y1',
 'y2',
 'y3',
 'y4',
 'y5',
 'y6',
 'y7',
 'y8',
 'y9',
 'y10',
 'y11',
 'y12',
 'y13',
 'y14',
 'y15',
 'y16',
 'y17',
 'y18',
 'y19',
 'y20',
 'y21',
 'y22',
 'y23',
 'y24']